### ResNet 50 output sizes of stages

In [4]:
# surpress UserWarnings
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [5]:
from mmdet.models import ResNet
import torch
res50 = ResNet(depth=50)
res50.eval()
inputs =  torch.rand(1, 3, 224, 224)
level_outputs = res50.forward(inputs)
for level_out in level_outputs:
    print(tuple(level_out.shape))

(1, 256, 56, 56)
(1, 512, 28, 28)
(1, 1024, 14, 14)
(1, 2048, 7, 7)


### adjust keys in checkpoint weights file

In [1]:
import torch
root_path = "../../../"
checkpoint = torch.load(f'{root_path}checkpoints/faster_rcnn_r50_fpn_1x_voc0712_20220320_192712-54bef0f3.pth')
state_dict = checkpoint['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("backbone.","backbone.resnet.")] = state_dict.pop(key)
checkpoint['state_dict'] = state_dict
torch.save(checkpoint, f'{root_path}checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth')

### Test inference FeedbackNet

In [ ]:
from mmdet.apis import init_detector, inference_detector
import mmcv

root_path = "../../../"
# Specify the path to model config and checkpoint file
config_file = f'{root_path}configs/pascal_voc/faster_rcnn_r50fbadd_fpn_1x_voc0712.py'
checkpoint_file = f'{root_path}checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth'

# build the model from a config file and a checkpoint file
#model = init_detector(config_file, checkpoint_file, device='cpu')
model = init_detector(config_file, checkpoint_file, device='cpu')

In [5]:
# will only load it once
img = mmcv.imread(f'{root_path}demo/demo.jpg')
result = inference_detector(model, img)
# visualize the results in a new window
model.show_result(img, result, out_file=f'{root_path}output/result.jpg')

### Model summary

In [ ]:
# Print model's state_dict
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

#print(model)
from torchsummary import summary
summary(model)#, (1, 3, 224, 224))

In [ ]:
from torchsummary import summary
summary(res50, (1, 3, 224, 224))

### Robustness Benchmark results

In [6]:
import mmcv
rob_fbmod = mmcv.load("evaluation/rob-bm_fb-mod.pkl")
rob_fbmod_results = mmcv.load("evaluation/rob-bm_fb-mod_results.pkl")

In [10]:
rob_fbmod_results
rob_fbmod_results['speckle_noise']

{0: [{'ap': 0.8611395},
  {'ap': 0.8624071},
  {'ap': 0.795072},
  {'ap': 0.71224385},
  {'ap': 0.67824864},
  {'ap': 0.8627916},
  {'ap': 0.88581085},
  {'ap': 0.8905615},
  {'ap': 0.66129816},
  {'ap': 0.8622946},
  {'ap': 0.75168365},
  {'ap': 0.8812101},
  {'ap': 0.87912333},
  {'ap': 0.8489476},
  {'ap': 0.85577387},
  {'ap': 0.56921345},
  {'ap': 0.8410044},
  {'ap': 0.80111444},
  {'ap': 0.85281754},
  {'ap': 0.768466}],
 1: [{'ap': 0.7506241},
  {'ap': 0.79606766},
  {'ap': 0.72707963},
  {'ap': 0.5402336},
  {'ap': 0.5848283},
  {'ap': 0.82268405},
  {'ap': 0.851375},
  {'ap': 0.7862084},
  {'ap': 0.6062227},
  {'ap': 0.8235061},
  {'ap': 0.715005},
  {'ap': 0.8281002},
  {'ap': 0.85062295},
  {'ap': 0.8408999},
  {'ap': 0.7835589},
  {'ap': 0.48582143},
  {'ap': 0.74332947},
  {'ap': 0.7526977},
  {'ap': 0.78142697},
  {'ap': 0.72074634}],
 2: [{'ap': 0.67074007},
  {'ap': 0.78501725},
  {'ap': 0.6480822},
  {'ap': 0.49208686},
  {'ap': 0.56198925},
  {'ap': 0.75929755},
  {'

In [3]:
rob_fbmod
rob_fbmod[0][14][1]

array([9.7176231e+01, 9.3941269e+00, 3.5300000e+02, 3.0582852e+02,
       2.6860243e-01], dtype=float32)

rob_fmod:

| 4952  |  20    |    n     |   5   |
|---|---|---|---|
|images |classes | detected bboxes | bbox + certainty |
                     

In [ ]:
""" 20 classes of PASCAL VOC
 0: aeroplane
 1: bicycle
 2: bird
 3: boat
 4: bottle
 5: bus
 6: car
 7: cat
 8: chair
 9: cow
10: diningtable
11: dog
12: horse
13: motorbike
14: person
15: pottedplant
16: sheep
17: sofa
18: train
19: tvmonitor
"""